In [10]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import scipy.stats as st
import sklearn.metrics as metrics
# test classification dataset
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score,roc_auc_score,recall_score,classification_report,mean_squared_error,accuracy_score,confusion_matrix


In [3]:
#for getting the columns only
test=pd.read_csv("cleaned_data_test_feature_engineered.csv")


#loading the data
data_train_smote=pd.read_csv("full_train_after_smote_adasyn.csv")
data_test_smote=pd.read_csv("cleaned_data_test_feature_engineered.csv")


#dropping the feature engineered data from test

data_test_smote=data_test_smote.drop(["YEAR","INCIDENT_ID","MONTH","DAY","DATE"],axis=1)
data_train_smote=data_train_smote.drop(["INCIDENT_ID"],axis=1)

print(data_train_smote.shape,data_test_smote.shape)


#marker for concatenation of data
data_train_smote["mark"]="train"
data_test_smote["mark"]="test"

full_test_train_data=pd.concat([data_train_smote,data_test_smote])


#check for the columns to exclude 
def feature_enginnering_a_into_2b(colnames,Xfull,number_of_columns,flag=1):
    for column in colnames:
        if (flag<number_of_columns):
            print(column,colnames[flag])
            a=column+"into"+colnames[flag]
            Xfull[a]=Xfull[column]*(2*Xfull[colnames[flag]])
            print(a)
            flag+=1
        else:
            flag+=1
            print("exceeded")
    return Xfull


#calling feature engineering function
colnames=full_test_train_data.columns.values
print(colnames)
p=len(colnames)
feature_engineered_dataframe_a_into_2b=feature_enginnering_a_into_2b(colnames=colnames,
                                                                    Xfull=full_test_train_data,
                                                                     number_of_columns=15)


print(feature_engineered_dataframe_a_into_2b.shape)

#sapretaing test and train data 
train_saperated=feature_engineered_dataframe_a_into_2b[feature_engineered_dataframe_a_into_2b["mark"]=="train"]
test_saperated=feature_engineered_dataframe_a_into_2b[feature_engineered_dataframe_a_into_2b["mark"]=="test"]
print(train_saperated.shape,test_saperated.shape)

#remocving unwanted columns
test_saperated.drop(["MULTIPLE_OFFENSE","MULTIPLE_OFFENSEintoX_1","mark"],axis=1,inplace=True)
train_saperated.drop(["MULTIPLE_OFFENSEintoX_1","mark"],axis=1,inplace=True)
print(train_saperated.shape,test_saperated.shape)


#splitting into train and validation
train,val=train_test_split(train_saperated,test_size=.20,random_state=42)
print(train.shape,val.shape)



#splitting into x and y
Xfull=train_saperated.drop(["MULTIPLE_OFFENSE"],axis=1)
yfull=train_saperated["MULTIPLE_OFFENSE"]
print(Xfull.shape,yfull.shape)


Xtrain=train.drop(["MULTIPLE_OFFENSE"],axis=1)
ytrain=train["MULTIPLE_OFFENSE"]
print(Xtrain.shape,ytrain.shape)

Xval=val.drop(["MULTIPLE_OFFENSE"],axis=1)
yval=val["MULTIPLE_OFFENSE"]
print(Xval.shape,yval.shape)


(45565, 16) (15903, 15)
['MULTIPLE_OFFENSE' 'X_1' 'X_10' 'X_11' 'X_12' 'X_13' 'X_14' 'X_15' 'X_2'
 'X_3' 'X_4' 'X_5' 'X_6' 'X_7' 'X_8' 'X_9' 'mark']
MULTIPLE_OFFENSE X_1
MULTIPLE_OFFENSEintoX_1
X_1 X_10
X_1intoX_10
X_10 X_11
X_10intoX_11
X_11 X_12
X_11intoX_12
X_12 X_13
X_12intoX_13
X_13 X_14
X_13intoX_14
X_14 X_15
X_14intoX_15
X_15 X_2
X_15intoX_2
X_2 X_3
X_2intoX_3
X_3 X_4
X_3intoX_4
X_4 X_5
X_4intoX_5
X_5 X_6
X_5intoX_6
X_6 X_7
X_6intoX_7
X_7 X_8
X_7intoX_8
exceeded
exceeded
exceeded
(61468, 31)
(45565, 31) (15903, 31)
(45565, 29) (15903, 28)
(36452, 29) (9113, 29)
(45565, 28) (45565,)
(36452, 28) (36452,)
(9113, 28) (9113,)


In [4]:
lrc = LogisticRegression(class_weight="balanced")
rfc=RandomForestClassifier(class_weight="balanced")
adbc=AdaBoostClassifier()
bgc=BaggingClassifier()
gnb = GaussianNB()
knn=KNeighborsClassifier()
dtc = DecisionTreeClassifier()
bgcl_lrc = BaggingClassifier(base_estimator=lrc, random_state=42)
ab_rfc = AdaBoostClassifier(base_estimator=rfc,random_state=42)
ab_dtc = AdaBoostClassifier(base_estimator=dtc,random_state=42)
ab_nbc=  AdaBoostClassifier(base_estimator=gnb,random_state=42)
ab_lrc=  AdaBoostClassifier(base_estimator=lrc,random_state=42)
gbc=GradientBoostingClassifier()
ab_gbc=  AdaBoostClassifier(base_estimator=gbc,random_state=42)
xgbc=XGBClassifier()
ab_xgbc=  AdaBoostClassifier(base_estimator=xgbc,random_state=42)
lgbc=LGBMClassifier(class_weight="balanced")

In [13]:

# models = [('lrc',LogisticRegression(class_weight="balanced")),
#           ('ab_gbc',AdaBoostClassifier(base_estimator=gbc,random_state=42)),
#           ('gbc',GradientBoostingClassifier())]
# stacking = StackingClassifier(estimators=models)

level0 = list()
level0.append(('ab_gbc',AdaBoostClassifier(base_estimator=gbc,random_state=42)))
level0.append(('gbc',GradientBoostingClassifier()))
# define meta learner model
level1 = LogisticRegression(class_weight="balanced")

model_stacked = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
model_stacked.fit(Xfull,yfull)

StackingClassifier(cv=5,
                   estimators=[('ab_gbc',
                                AdaBoostClassifier(algorithm='SAMME.R',
                                                   base_estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                                                             criterion='friedman_mse',
                                                                                             init=None,
                                                                                             learning_rate=0.1,
                                                                                             loss='deviance',
                                                                                             max_depth=3,
                                                                                             max_features=None,
                                                                                          

In [17]:
predicted_final=model_stacked.predict(test_saperated)
predictions=pd.concat([test['INCIDENT_ID'],pd.DataFrame(predicted_final,columns=['MULTIPLE_OFFENSE'])],1)
# predictions=pd.concat([test['INCIDENT_ID'],pd.DataFrame(y_test_pred,columns=['MULTIPLE_OFFENSE']),1)
predictions.to_csv("predicted_stacked.csv",index=False)